In [6]:
import pandas as pd
import os

# --- SETUP: Generate Dummy Data if missing ---
if not os.path.exists("dummy_data.csv"):
    print("⚠️ dummy_data.csv not found. Creating it...")
    data_clf = {
        'age': [22, 25, 47, 52, 46, 56, 21, 23, 50, 20],
        'salary': [20000, 25000, 50000, 60000, 52000, 80000, 18000, 22000, 65000, 15000],
        'bought': [0, 0, 1, 1, 1, 1, 0, 0, 1, 0]
    }
    pd.DataFrame(data_clf).to_csv("dummy_data.csv", index=False)
    print("✅ Created dummy_data.csv")

if not os.path.exists("dummy_reg.csv"):
    print("⚠️ dummy_reg.csv not found. Creating it...")
    data_reg = {
        'years_experience': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        'level': [1, 1, 2, 2, 3, 3, 4, 4, 5, 5],
        'salary': [30000, 35000, 50000, 60000, 80000, 85000, 100000, 110000, 130000, 150000]
    }
    pd.DataFrame(data_reg).to_csv("dummy_reg.csv", index=False)
    print("✅ Created dummy_reg.csv")
# ---------------------------------------------

In [7]:
import pandas as pd
import numpy as np
import etna

# 1. Test Classification (Auto-detect should work fine here)
print("=== TEST 1: CLASSIFICATION ===")
model = etna.Model("dummy_data.csv", target="bought")
model.train(epochs=200, lr=0.1)
print(f"✅ Predictions: {model.predict()}")

# 2. Test Regression (Manually specifying task_type="regression")
print("\n=== TEST 2: REGRESSION ===")
# Note: We pass task_type="regression" to skip the auto-detect logic for this tiny dataset
model_reg = etna.Model("dummy_reg.csv", target="salary", task_type="regression")
model_reg.train(epochs=500, lr=0.001) # More epochs for regression accuracy
preds_reg = model_reg.predict()

# Formatting for clean output
print(f"✅ Predictions: {[round(p, 1) for p in preds_reg]}")

=== TEST 1: CLASSIFICATION ===
[*] Auto-Detected Task: Classification (Target 'bought')
⚙️  Preprocessing data...
🚀 Initializing Rust Core [In: 2, Out: 2]...
🔥 Training started (Optimizer: SGD)...
✅ Training complete!
✅ Predictions: [np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(0)]

=== TEST 2: REGRESSION ===
[*] User Task: Regression (Target 'salary')
⚙️  Preprocessing data...
🚀 Initializing Rust Core [In: 2, Out: 1]...
🔥 Training started (Optimizer: SGD)...
✅ Training complete!
✅ Predictions: [29888.3, 33942.6, 53412.8, 58065.2, 77461.9, 86439.6, 102715.8, 115647.4, 131219.6, 144555.4]


In [8]:
# 3. Test Save & Track (New Feature)
print("\n=== TEST 3: SAVE & TRACK EXPERIMENT ===")

# We will save the regression model we just trained
print("💾 Saving model artifact and logging metrics to MLflow...")

# This one function call does everything:
# 1. Saves 'mvp_regressor.json' locally
# 2. Logs the file to MLflow Artifacts
# 3. Logs the loss curve to MLflow Metrics
model_reg.save_model(
    "mvp_regressor.json",
    run_name="MVP_Demo_Run",
    mlflow_tracking_uri="http://localhost:5000" # Explicitly pass the URI
)

print("✅ Success! Check the dashboard at http://localhost:5000")


=== TEST 3: SAVE & TRACK EXPERIMENT ===
💾 Saving model artifact and logging metrics to MLflow...
Saving model to mvp_regressor.json...
Logging to MLflow at http://localhost:5000...


2026/01/18 15:34:09 INFO mlflow.tracking.fluent: Experiment with name 'ETNA_Experiments' does not exist. Creating a new experiment.


🏃 View run MVP_Demo_Run at: http://localhost:5000/#/experiments/1/runs/ba388b8350474f6ebc92273a2ca66416
🧪 View experiment at: http://localhost:5000/#/experiments/1
Model saved & tracked!
✅ Success! Check the dashboard at http://localhost:5000


In [9]:
# 4. Test Loading (New Feature)
print("\n=== TEST 4: LOAD MODEL ===")

# Load the model we just saved in Test 3
print("📂 Loading 'mvp_regressor.json' back into memory...")
loaded_model = etna.Model.load("mvp_regressor.json")

# --- MVP WORKAROUND: Restore Python State ---
loaded_model.preprocessor = model_reg.preprocessor
loaded_model.task_type = model_reg.task_type
loaded_model.target = model_reg.target
# --------------------------------------------

print(f"✅ Model loaded successfully!")
print(f"   - Task Type: {loaded_model.task_type}")
print(f"   - Backend: {loaded_model.rust_model}")

# Verify it predicts correctly
print("🔮 Running prediction with loaded model...")
new_preds = loaded_model.predict(data_path="dummy_reg.csv")

# Compare with the previous results (sanity check)
print(f"✅ Loaded Predictions: {[round(p, 1) for p in new_preds[:10]]}")


=== TEST 4: LOAD MODEL ===
📂 Loading 'mvp_regressor.json' back into memory...
📂 Loading model from mvp_regressor.json...
✅ Model loaded successfully!
✅ Model loaded successfully!
   - Task Type: regression
   - Backend: <builtins.EtnaModel object at 0x000001BB27AD3550>
🔮 Running prediction with loaded model...
Transforming input data...
✅ Loaded Predictions: [29888.3, 33942.6, 53412.8, 58065.2, 77461.9, 86439.6, 102715.8, 115647.4, 131219.6, 144555.4]
